In [2]:
# from huggingface_hub import notebook_login
from transformers import WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from datasets import load_dataset, DatasetDict, concatenate_datasets
from datasets import Audio
from transformers import WhisperForConditionalGeneration
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer, pipeline
from accelerate import PartialState, prepare_pippy
from accelerate.utils import set_seed

from torch.distributed.pipelining import Pipe

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# notebook_login()

In [3]:
# model_name = "openai/whisper-small"
model_name = "openai/whisper-large-v3-turbo"
cache_dir = '/mnt/8THDD0/storage/huggingface'
set_seed(42)
target_lang = 'yue'
# model_name = "openai/whisper-large-v3-turbo"
output_dir = "/mnt/8THDD0/storage/train/output/"+ model_name.split('/')[1] +"-" + target_lang

In [6]:
tokenizer = WhisperTokenizer.from_pretrained(model_name, language=target_lang, task="transcribe", cache_dir=cache_dir)

In [4]:
pipe = pipeline(task="automatic-speech-recognition", model=output_dir+'/checkpoint-30000', tokenizer=tokenizer)
try:
    pipe.save_pretrained(output_dir + '/final')
except:
    pipe.save_pretrained(output_dir + '/result')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[2024-11-21 09:28:23,966] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jacky/anaconda3/envs/whisper/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jacky/anaconda3/envs/whisper/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-11.8/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jacky/anaconda3/envs/whisper/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-11.8/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jacky/anaconda3/envs/whisper/compiler_compat/ld: /usr/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/jacky/anaconda3/envs/whisper/compiler_compat/ld: /usr/local/cuda-11.8/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/jacky/anaconda3/envs/whisper/compiler_compat/ld: /usr/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/home/jacky/anaconda3/envs/whisper

In [ ]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

# torch.setnum_threads(1)
common_voice = DatasetDict()

common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test", cache_dir = cache_dir)

common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

processor = WhisperProcessor.from_pretrained(output_dir + "/final/")
model = WhisperForConditionalGeneration.from_pretrained(output_dir + "/final/").to("cuda")

def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

result = common_voice.map(map_to_pred)


en_train_0.tar:   0%|          | 0.00/1.85G [00:00<?, ?B/s]

en_train_1.tar:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

en_train_2.tar:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

en_train_3.tar:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

en_train_4.tar:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

en_train_5.tar:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

en_train_6.tar:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

en_train_7.tar:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

en_train_8.tar:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [6]:
cer = load("cer")
print(100 * cer.compute(references=result['test']['reference'], predictions=result['test']["prediction"]))

7.548162538449085


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from transformers import pipeline
from huggingface_hub import notebook_login
import os

# Define the path to your local model directory
model_path = output_dir + '/final/'

# Define the name of your repository on Hugging Face (must include username)
repo_name = 'JackyHoCL/whisper-large-v3-turbo-cantonese-yue-english'

# Push the model to the repository
pipeline("automatic-speech-recognition", model=model_path).push_to_hub(repo_name)

print(f"Model successfully uploaded to {repo_name}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


README.md:   0%|          | 0.00/513 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

Model successfully uploaded to JackyHoCL/whisper-large-v3-turbo-cantonese-yue-english


In [1]:
!ct2-transformers-converter --model JackyHoCL/whisper-large-v3-turbo-cantonese-yue-english --output_dir /mnt/8THDD0/storage/train/output/ct2/whisper-large-v3-turbo-cantonese-yue-english-ct2 --copy_files tokenizer.json preprocessor_config.json --quantization float16

model.safetensors: 100%|███████████████████| 3.24G/3.24G [02:17<00:00, 23.5MB/s]
